In [1]:
import numpy as np
import pandas as pd

In [2]:
def donchian(prices, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame(prices.copy())
    df['upr'] = df.high.rolling(period).max().shift(periods=1)
    df['lwr'] = df.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df


def remainder_zero(series, divisor):
    '''Increment number until remainder is zero.'''
    def increment_dividend(dividend, divisor):
        '''Increment dividend while remainder does not equal zero.'''
        while dividend % divisor != 0:
            dividend += 1
        return dividend
    series = series.map(lambda x: increment_dividend(x, divisor))
    return series


def weekly(exchange, tidm):
    '''Generate weekly prices from SharePad csv file of daily prices.'''
    df = pd.read_csv(
        f'{exchange}_{tidm}_prices.csv',
        header=0,
        names=['date', 'open', 'high', 'low', 'close'],
        index_col=0,
        usecols=[0, 1, 2, 3, 4],
        parse_dates=True,
        dayfirst=True,
    )
    df = df.sort_index()
    functions = dict(open='first', high='max', low='min', close='last')
    df = df.resample('W-FRI').agg(functions)
    df = df / 100
    return df

In [3]:
# Trade parameters.
exchange = 'LSE'
tidm = 'HSV'
periods = [48, 24, 12, 6]  # System look back periods.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2  # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [4]:
# Import weekly prices.
prices = weekly(exchange, tidm)

In [5]:
# Donchian channels & trade signals.
donchian_signals = []
for sys, period in enumerate(periods):
    dc = donchian(prices, period)
    dc['sys'] = sys
    dc['period'] = period
    if sys == 0:
        dc['entry'] = np.where(dc.close > dc.upr, 1, 0)
    else:
        dc['entry'] = buy
    dc['exit'] = np.where(dc.close < dc.mid, 1, 0)
    dc['state'] = 0
    for i in range(period, len(dc)):
        if dc.loc[dc.index[i], 'entry'] == 1 and dc.loc[dc.index[i - 1], 'state'] == 0:
            dc.loc[dc.index[i], 'state'] = 1
        elif dc.loc[dc.index[i], 'exit'] == 1:
            dc.loc[dc.index[i], 'state'] = 0
        else:
            dc.loc[dc.index[i], 'state'] = dc.loc[dc.index[i - 1], 'state']
    dc['buy'] = np.where(np.logical_and(dc.state == 1, dc.state.shift(periods=1) == 0), 1, 0)
    buy = dc.buy
    dc['sell'] = np.where(np.logical_and(dc.state == 0, dc.state.shift(periods=1) == 1), 1, 0)
    donchian_signals.append(dc)

In [6]:
# Trade list.
db = pd.DataFrame(donchian_signals[0].copy())
db = db[db.buy == 1]
# db['volatility'] = abs((db.mid - db.close) / db.close)
# db['risk_amt'] = (position_size * risk_pct) / db.volatility
# db['shares'] = (db.risk_amt / db.close).astype('int')
# db['_shares'] = remainder_zero(db.shares, len(periods)) # Adjust shares to be divisible by number of systems.
# db['_risk_amt'] = db.close * db.shares # Adjust risk amount for revised share count.
db

,open,high,low,close,upr,lwr,mid,sys,period,entry,exit,state,buy,sell
date,,,,,,,,,,,,,,
2001-12-07,1.1954,1.2277,1.1954,1.2277,1.2062,0.87769,1.041945,0,48,1,0,1,1,0
2003-09-05,1.1254,1.1738,1.1200,1.1706,1.1415,0.78615,0.963825,0,48,1,0,1,1,0
2009-09-11,3.1898,3.4397,3.1683,3.3514,3.2997,1.77910,2.539400,0,48,1,0,1,1,0
2011-05-06,5.2898,5.4492,5.2834,5.4277,5.4062,4.21080,4.808500,0,48,1,0,1,1,0
2013-05-24,2.3843,2.8571,2.3843,2.7462,2.7085,1.56260,2.135550,0,48,1,0,1,1,0
2014-01-31,3.0563,3.5646,3.0369,3.5280,3.3385,1.99120,2.664850,0,48,1,0,1,1,0
2016-05-27,4.1760,4.9320,4.1760,4.8800,4.7977,3.56100,4.179350,0,48,1,0,1,1,0
2018-05-25,8.4550,9.1450,7.9350,8.9200,8.7200,6.82500,7.772500,0,48,1,0,1,1,0
2019-04-05,10.2600,11.3500,10.2600,10.9500,10.6600,7.27500,8.967500,0,48,1,0,1,1,0


In [8]:
# Position size (sell).
trades = []
for donchian in donchian_signals:
    ds = pd.DataFrame(donchian.copy())
    ds = ds[ds.sell == 1]
    dp = pd.concat([db, ds])
    dp = dp.sort_index()
    trades.append(dp)
trades[0]

,open,high,low,close,upr,lwr,mid,sys,period,entry,exit,state,buy,sell
date,,,,,,,,,,,,,,
2001-12-07,1.1954,1.2277,1.1954,1.2277,1.2062,0.87769,1.041945,0,48,1,0,1,1,0
2002-09-06,1.1792,1.1846,1.1415,1.1469,1.3591,0.99077,1.174935,0,48,0,1,0,0,1
2003-09-05,1.1254,1.1738,1.1200,1.1706,1.1415,0.78615,0.963825,0,48,1,0,1,1,0
2007-07-27,3.8769,3.9415,3.5668,3.5969,4.3917,3.41600,3.903850,0,48,0,1,0,0,1
2009-09-11,3.1898,3.4397,3.1683,3.3514,3.2997,1.77910,2.539400,0,48,1,0,1,1,0
2010-11-12,4.6986,4.7148,4.3745,4.3938,5.4062,3.41600,4.411100,0,48,0,1,0,0,1
2011-05-06,5.2898,5.4492,5.2834,5.4277,5.4062,4.21080,4.808500,0,48,1,0,1,1,0
2011-08-05,5.2231,5.3168,4.6814,4.8623,5.7615,4.36580,5.063650,0,48,0,1,0,0,1
2013-05-24,2.3843,2.8571,2.3843,2.7462,2.7085,1.56260,2.135550,0,48,1,0,1,1,0
